In [1]:
import pandas as pd

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
from nltk.corpus import stopwords


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('data/judge-1377884607_tweet_product_company.csv', encoding = 'unicode_escape')


In [3]:
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product


In [4]:
#Converting target to int values

df['target'] = df.is_there_an_emotion_directed_at_a_brand_or_product
df.target.replace(to_replace = "Positive emotion", value = 1, inplace= True)
df.target.replace(to_replace = "Negative emotion", value = 0, inplace= True)
df.target.replace(to_replace = "No emotion toward brand or product", value = 0, inplace= True)
df.target.replace(to_replace = "I can't tell", value = 0, inplace= True)
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,target
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,0
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,1
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,1
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,0
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,1
...,...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion,1
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product,0
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product,0
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product,0


In [5]:
df

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,target
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,0
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,1
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,1
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,0
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,1
...,...,...,...,...
9088,Ipad everywhere. #SXSW {link},iPad,Positive emotion,1
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product,0
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product,0
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product,0


In [6]:
df.iloc[9091].tweet_text

'Some Verizon iPhone customers complained their time fell back an hour this weekend.  Of course they were the New Yorkers who attended #SXSW.'

In [7]:
df.emotion_in_tweet_is_directed_at.value_counts()

iPad                               946
Apple                              661
iPad or iPhone App                 470
Google                             430
iPhone                             297
Other Google product or service    293
Android App                         81
Android                             78
Other Apple product or service      35
Name: emotion_in_tweet_is_directed_at, dtype: int64

In [8]:
#First question, do we collapse this into Apple vs. Android/Google?

In [9]:
df.emotion_in_tweet_is_directed_at.isna().sum()

5802

In [10]:
#That's over half the dataset.


In [11]:
df.emotion_in_tweet_is_directed_at.isna()

0       False
1       False
2       False
3       False
4       False
        ...  
9088    False
9089     True
9090     True
9091     True
9092     True
Name: emotion_in_tweet_is_directed_at, Length: 9093, dtype: bool

In [12]:
nandf = df[df.emotion_in_tweet_is_directed_at.isna()]

In [13]:
nandf

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,target
5,@teachntech00 New iPad Apps For #SpeechTherapy...,NaN,No emotion toward brand or product,0
6,NaN,NaN,No emotion toward brand or product,0
16,Holler Gram for iPad on the iTunes App Store -...,NaN,No emotion toward brand or product,0
32,"Attn: All #SXSW frineds, @mention Register fo...",NaN,No emotion toward brand or product,0
33,Anyone at #sxsw want to sell their old iPad?,NaN,No emotion toward brand or product,0
...,...,...,...,...
9087,"@mention Yup, but I don't have a third app yet...",NaN,No emotion toward brand or product,0
9089,"Wave, buzz... RT @mention We interrupt your re...",NaN,No emotion toward brand or product,0
9090,"Google's Zeiger, a physician never reported po...",NaN,No emotion toward brand or product,0
9091,Some Verizon iPhone customers complained their...,NaN,No emotion toward brand or product,0


In [14]:
df.is_there_an_emotion_directed_at_a_brand_or_product.value_counts()

No emotion toward brand or product    5389
Positive emotion                      2978
Negative emotion                       570
I can't tell                           156
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [15]:
nandf.is_there_an_emotion_directed_at_a_brand_or_product.value_counts()

No emotion toward brand or product    5298
Positive emotion                       306
I can't tell                           147
Negative emotion                        51
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [16]:
allprodf = df[df.emotion_in_tweet_is_directed_at.notnull()]

In [17]:
allprodf

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,target
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion,0
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion,1
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion,1
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion,0
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion,1
...,...,...,...,...
9077,@mention your PR guy just convinced me to swit...,iPhone,Positive emotion,1
9079,&quot;papyrus...sort of like the ipad&quot; - ...,iPad,Positive emotion,1
9080,Diller says Google TV &quot;might be run over ...,Other Google product or service,Negative emotion,0
9085,I've always used Camera+ for my iPhone b/c it ...,iPad or iPhone App,Positive emotion,1


In [18]:
allprodf.is_there_an_emotion_directed_at_a_brand_or_product.value_counts()

Positive emotion                      2672
Negative emotion                       519
No emotion toward brand or product      91
I can't tell                             9
Name: is_there_an_emotion_directed_at_a_brand_or_product, dtype: int64

In [19]:
df.tweet_text.isna().sum()

1

In [20]:
ttna = df[df.tweet_text.isna()]
ttna

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product,target
6,NaN,NaN,No emotion toward brand or product,0


Okay, this is just a whole null row. Dropping it.

In [21]:
df.tweet_text.dropna(inplace=True)

While programming got an error related to a tweet not being a string. Going to force that. 

In [22]:
df.tweet_text = df.tweet_text.map(str)


Some conclusions from this little EDA: We have many more samples of positive than negative emotions. Tweets rated as being about a product are much more likely to be rated with some emotion or another

In [23]:
#Recycling some code for stemming and lemming


sw = stopwords.words('english')

def doc_preparer(doc, stop_words=sw):
    '''
    
    :param doc: a document from the satire corpus 
    :return: a document string with words which have been 
            lemmatized, 
            parsed for stopwords, 
            made lowercase,
            and stripped of punctuation and numbers.
    '''
    
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    doc = regex_token.tokenize(doc)
    doc = [word.lower() for word in doc]
    doc = [word for word in doc if word not in sw]
    print(doc)
    doc = pos_tag(doc)
    doc = [(word[0], get_wordnet_pos(word[1])) for word in doc]
    lemmatizer = WordNetLemmatizer() 
    doc = [lemmatizer.lemmatize(word[0], word[1]) for word in doc]
    return ' '.join(doc)




cv = CountVectorizer()


### Norman's code for nltk lemmer in sklearn

 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, text):
        return [self.wnl.lemmatize(t) for t in word_tokenize(text)]

vectorizer = CountVectorizer(tokenizer=LemmaTokenizer,
                       stop_words = 'english',
                       lowercase = True,
                       token_pattern = r"([a-zA-Z]+(?:’[a-z]+)?)",
                       max_df = 2,
                       min_df = 20)


In [24]:
test = doc_preparer(df.tweet_text[0])

print(df.tweet_text[0])
print(test)

.@wesley83 I have a 3G iPhone. After 3 hrs tweeting at #RISE_Austin, it was dead!  I need to upgrade. Plugin stations at #SXSW.
wesley g iphone hrs tweeting rise austin dead need upgrade plugin stations sxsw


Okay, test results.

Stripping @, #, and numbers from tweets seems more harmful to semantic meaning than from regular speech. OTOH, SXSW and #SXSW seem very similar, should probably be collapsed into one? 

Dropping all @*** might be good, unless it's directly @-ing the client or the client's competition.

In [25]:
vectorizer = CountVectorizer()

In [26]:
#

In [27]:
X = df.tweet_text
y = df.target

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =42)

In [29]:
type(X_train)

pandas.core.series.Series

In [30]:
y_train.value_counts()

0    4582
1    2237
Name: target, dtype: int64

In [31]:
# x_train_prepped = X_train.map(doc_preparer)
# .map doesn't want to work here. Dirty work around, the for loop!
typelist = []
for i in X_train:
    typelist.append(type(i))
    
print(typelist.count(str))
len(X_train)

6819


6819

There is one tweet that is not a string. Great.

In [32]:
typelist = []
for i in df.tweet_text:
    typelist.append(type(i))
    
print(typelist.count(str))
len(df.tweet_text)

9093


9093

In [33]:
x_train_prepped = X_train.map(doc_preparer)

x_train_prepped

3312    good design means getting simple mention talk ...
1302    wishing sxsw see rumored demo mention new soci...
5344    rt mention problems time stop link edchat muse...
4580    mention new ipad apps speech therapy communica...
3094                            ipad queue epic sxsw link
                              ...                        
5734    rt mention many asked last night free android ...
5191    rt mention quot google quot refrigerator magne...
5390    rt mention android developers friends let hang...
860     geeky love rt mention apple opening temporary ...
7270    anyone know iphone developers attending sxsw p...
Name: tweet_text, Length: 6819, dtype: object

In [34]:
x_train_defaltvec = vectorizer.fit_transform(x_train_prepped)

In [35]:
x_train_defaltvec

<6819x8023 sparse matrix of type '<class 'numpy.int64'>'
	with 77567 stored elements in Compressed Sparse Row format>

In [36]:
basicmodel = MultinomialNB()

In [37]:
basicmodel.fit(X = x_train_defaltvec, y = y_train)

MultinomialNB()

In [52]:
cvdict = {}
def reportcrossval(model, modelname, xtrain, ytrain):
    print(cross_val_score(model, xtrain, ytrain))
    cvmean = cross_val_score(model, xtrain, ytrain).mean()
    cvdict[modelname] = cvmean 
    print(f"The cv mean of {modelname} is {cvmean}") 

Baseline is 

In [44]:
df.target.value_counts()

4546.5

In [43]:
baseline = 6115/(6115+2978)
baseline 

0.6724953260750027

In [53]:
reportcrossval(basicmodel, "basicmodel", x_train_defaltvec, y_train)

[0.70527859 0.70747801 0.69208211 0.6957478  0.68525312]
The cv mean of basicmodel is 0.6971679256771439


In [55]:
ngram1to3 = CountVectorizer(ngram_range=(1, 3))

x_train_ngrams = ngram1to3.fit_transform(x_train_prepped)

ngrammodel = MultinomialNB()

ngrammodel.fit(x_train_ngrams, y_train)



MultinomialNB()

In [56]:
reportcrossval(ngrammodel, "ngrammodel", x_train_ngrams, y_train)

[0.65615836 0.65395894 0.65322581 0.6414956  0.64416728]
The cv mean of ngrammodel is 0.6498011975481031
